In [1]:
from dotenv import load_dotenv
import pandas as pd
import csv
import os
import base64
import requests
import json
from pprint import pprint
from ids import id
from ids import secret

#For Genius API

import numpy as np
import lyricsgenius


#Import natural language toolkit
import nltk

In [2]:
load_dotenv()
client_id = id
client_secret = secret

# Asegurarse de que los Id's están presentes, porque si no, la función get token da error.
# print(client_id, client_secret)

In [3]:
# Definiendo funciones <3

def get_token():
    auth_string = client_id + ":" + client_secret
    auth_bytes = auth_string.encode("utf-8")
    auth_base64 = str(base64.b64encode(auth_bytes), "utf-8")

    url = "https://accounts.spotify.com/api/token"
    headers = {
        "Authorization" : "Basic " + auth_base64,
        "Content-Type" : "application/x-www-form-urlencoded"
    }

    data = {"grant_type" : "client_credentials"}

    result = requests.post(url, headers = headers, data = data)
    json_response = json.loads(result.content)
    token = json_response["access_token"]
    return token

def get_auth_header(token):
    return{"Authorization" : "Bearer " + token}

def search_for_playlist(token, playlist_name):
    url = "https://api.spotify.com/v1/search"
    headers = get_auth_header(token)
    query = f"?q={playlist_name}&type=playlist&limit=1"
    query_url = url + query
    result = requests.get(query_url, headers = headers)
    json_result_playlist = json.loads(result.content)["playlists"]["items"]
    if len(json_result_playlist) == 0:
        print("No playlist with this name found...")
        return None
    
    # pprint(json_result_playlist)
    return json_result_playlist[0]

def get_songs_in_playlist(token, artist_id):
    url = f"https://api.spotify.com/v1/playlists/{playlist_id}"
    headers = get_auth_header(token)
    result = requests.get(url, headers = headers)
    json_result = json.loads(result.content)["tracks"]["items"]
    # pprint(json_result)
    return json_result


In [4]:
token = get_token()
result = search_for_playlist(token, "Top Canciones 2019")
playlist_id = result["id"]
songs = get_songs_in_playlist(token, playlist_id)
song_list = []
for idx, song in enumerate(songs):
    # print(f'{idx + 1}. {song["track"]["name"]}')
    song_data = {"Song":song["track"]["name"],
                 "Artist":song["track"]["artists"][0]["name"],
                 "Song Lenght (ms)":song["track"]["duration_ms"],
                 "Popularity":song["track"]["popularity"],
                 "Release date":song["track"]["album"]["release_date"],
                 "Song Id":song["track"]["id"]}
    song_list.append(song_data)
# pprint(song_list)
# pprint(songs)
spoti_19 = pd.DataFrame(song_list)
spoti_19.head()

,Song,Artist,Song Lenght (ms),Popularity,Release date,Song Id
0,Calma - Remix,Pedro Capó,238200,77,2018-10-05,5iwz1NiezX7WWjnCgY5TH4
1,Con Calma,Daddy Yankee,193226,83,2019-01-24,5w9c2J52mkdntKOmRLeM2m
2,Te Vi,Piso 21,231848,81,2018-12-14,059bcIhyc2SBwm6sw2AZzd
3,Otro Trago,Sech,225933,75,2019-04-19,1Ej96GIBCTvgH7tNX1r3qr
4,Callaita,Bad Bunny,250533,80,2019-05-31,2TH65lNHgvLxCKXM3apjxI


In [5]:
#Tomar la columna Song Id del DataFrame con los Top Tracks y convertir los valores en una lista de strings separadas por comas, para pasarlo así al request de los Audio Features

id_list = spoti_19["Song Id"]
id_list = ",".join(id_list)
id_list


'5iwz1NiezX7WWjnCgY5TH4,5w9c2J52mkdntKOmRLeM2m,059bcIhyc2SBwm6sw2AZzd,1Ej96GIBCTvgH7tNX1r3qr,2TH65lNHgvLxCKXM3apjxI,0s5DNDV6LAKn2u2Ufa3VjQ,1LIqNVhaIgxXUt2eVoIQJ3,5W83ErFkO3aKAIS1WMi6u0,0fea68AdmYNygeTGI4RC18,6FyRXC8tJUh863JCkyWqtk,7KbF6AdprOXEEHlsq11Z6d,1c0hsvHLELX6y8qymnpLKL,2qG5sZ7Si6sdK74qLxedYM,46lvmzK8wxAy66tjzXXSh0,2ksOAxtIxY8yElEWw8RhgK,2beS6dD5rovBxSA1zYoSHR,7g8YaUQABMal0zWe7a2ijz,3f5lA8nnc4YYF1qf78WpHw,5BDP2tky8oMQJPS33frKVp,3gE4eQH3K83Sght0ZLvuBK,6WkJ2OK163XXS2oARUC9JM,3KkXRkHbMCARz0aVfEt68P,6ocbgoVGwYJhOv1GgI9NsF,0TK2YIli7K1leLovkQiNik,2Fxmhks0bxGSBdJ92vM42m,6o8ogTrSX8H0F66Pdz3Tsc,7hbk4BaF66O09R1gq0bKwj,4lzxJ4jCuFDXXGkE1LmpKR,5PXsZaT8cOxb4mXrA20PMQ,4PJ3umtW1aWdfh5EITHc6T,0KHhNlHN5lWn7q332PiPys,0KoiMHhqKLoL4xM54rcWYY,25ZAibhr3bdlMCLmubZDVt,0akyEssGRVHstqCSWXusJL,4THusowxdHxwt0HBh5pyRx,32lexhcz93AinKK0qLTZaZ,1LiN0Z98FkR1t0m8KmLcAH,1nocRtwyNPVtGcIJqfgdzZ,7BuHdm4lQqcyG3qTwdkxus,4Clmg1g8n2yBYG13Z1oSdG,4yVO0wOhXsZC1RuPgHP524,2LIh4uzqq9cXMPzzmcToHl,1mT47pKLXUwPYGwB91GcNz,6De0lHrwBf

In [6]:
#Request para los Audio Features

def get_AF_in_songs(token, id_list):
    url = f"https://api.spotify.com/v1/audio-features?ids={id_list}"
    headers = get_auth_header(token)
    response = requests.get(url, headers = headers)
    response_json = response.json()
    #pprint(response_json)
    return response_json

In [7]:
token = get_token()
audio_features_data = get_AF_in_songs(token, id_list)["audio_features"]
audio_features_list = []


for idx, af in enumerate(audio_features_data):
    audio_features = {"Acousticness":af["acousticness"],
                 "Danceability":af["danceability"],
                 "Energy":af["energy"],
                 "Instrumentalness":af["energy"],
                 "Liveness":af["energy"],
                 "Speechiness":af["speechiness"],
                 "Tempo":af["tempo"],
                 "Valence":af["valence"]}
    audio_features_list.append(audio_features)
audio_features_df = pd.DataFrame(audio_features_list)
#audio_features_df

spoti_19_af_df = pd.concat([spoti_19, audio_features_df], axis = 1)
spoti_19_af_df

,Song,Artist,Song Lenght (ms),Popularity,Release date,Song Id,Acousticness,Danceability,Energy,Instrumentalness,Liveness,Speechiness,Tempo,Valence
0,Calma - Remix,Pedro Capó,238200,77,2018-10-05,5iwz1NiezX7WWjnCgY5TH4,0.3230,0.826,0.773,0.773,0.773,0.0524,126.899,0.761
1,Con Calma,Daddy Yankee,193226,83,2019-01-24,5w9c2J52mkdntKOmRLeM2m,0.1100,0.737,0.860,0.860,0.860,0.0593,93.989,0.656
2,Te Vi,Piso 21,231848,81,2018-12-14,059bcIhyc2SBwm6sw2AZzd,0.0446,0.877,0.777,0.777,0.777,0.1170,102.020,0.706
3,Otro Trago,Sech,225933,75,2019-04-19,1Ej96GIBCTvgH7tNX1r3qr,0.1360,0.746,0.700,0.700,0.700,0.3410,176.044,0.619
4,Callaita,Bad Bunny,250533,80,2019-05-31,2TH65lNHgvLxCKXM3apjxI,0.6000,0.610,0.624,0.624,0.624,0.3090,176.169,0.244
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,Sucker,Jonas Brothers,181026,84,2019-06-07,22vgEDb5hykfaTwLuskFGD,0.0427,0.842,0.734,0.734,0.734,0.0588,137.958,0.952
96,Cuando Nadie Ve,Morat,219586,82,2018-10-25,71iAechwVKGu26pahzFL0k,0.1330,0.806,0.778,0.778,0.778,0.0616,95.993,0.840
97,Que Le De,Rauw Alejandro,225600,60,2019-01-25,08aYFNUTIOMGq93e2VSArQ,0.1900,0.770,0.750,0.750,0.750,0.0785,99.997,0.609
98,Ponle,Rvssian,164764,67,2018-09-06,28O1p0bQXH2F6Qn8jBZZR4,0.1880,0.857,0.772,0.772,0.772,0.1950,94.030,0.589


In [8]:
client_access_token="KKVSlBN3kbpuuRtePodUd-RTz3CVxexH7EOatwekQVhbjLwWAHNSsiT9Mbf2vYMi"
LyricsGenius = lyricsgenius.Genius(client_access_token)

lyrics_df = spoti_19_af_df.Song
lyrics_df = pd.DataFrame(lyrics_df)
lyrics_df.head(10)



,Song
0,Calma - Remix
1,Con Calma
2,Te Vi
3,Otro Trago
4,Callaita
5,Desconocidos
6,HP
7,Secreto
8,LA CANCIÓN
9,Adan y Eva


In [9]:
# The package got some timeout issue so these two lines are needed. If you don't then there will be error when you scrape
# Source: https://github.com/johnwmillr/LyricsGenius/issues/121
LyricsGenius.timeout = 15  #timeout
LyricsGenius.sleep = 5

In [10]:
#Quick test first song

sample_song = lyrics_df.Song[0]
print(f"Name of the first song in the database is: {sample_song}")

# Search for the song's lyric
searched_song = LyricsGenius.search_song(sample_song)
print(f"The lyric is:\n{searched_song.lyrics}")

Name of the first song in the database is: Calma - Remix
Searching for "Calma - Remix"...
Done.
The lyric is:
24 ContributorsCalma (Remix) Lyrics[Letra de "Calma (Remix)"]

[Intro: Pedro Capó & Farruko]
Laramercy gang
Welcome to the paradise
¡Farru!
Cuatro abrazos y un café
Apenas me desperté
Y al mirarte recordé
Que ya todo lo encontré
Tu mano en mi mano
De todo escapamos
Juntos ver el sol caer
(Let them enjoy the paradise)

[Coro: Pedro Capó]
Vamos pa' la playa, pa’ curarte el alma
Cierra la pantalla, abre la Medalla
Todo el mar Caribe, viendo tu cintura
Tú le coqueteas, tú eres busca bullas y me gusta

[Post-Coro: Pedro Capó]
Lento y contento, cara al vie-ento
Lento y contento, cara al vie-ento

[Verso 1: Farruko]
Ya, ya, ya, ¡Farru!
Y aprovecha que el sol 'tá caliente y vamo' a disfrutar el ambiente
Vámono’ a meternos pa'l agua pa' que veas qué rico se siente
Y vámono' en tropical, por to'a la costa a chinchorrear
De chinchorro a chinchorro paramo' a darno' una Medalla
Bien fría, p

In [11]:
# Create an array to store each song's lyric
lyrics_arr = []

# Traverse through the database, get the song's lyrics from title, and do some preprocessing
for i in range(len(lyrics_df)):
    # get title
    song_title = lyrics_df.Song[i]
    
    # search for song in genius.com
    searched_song = LyricsGenius.search_song(song_title)
    
    # if we can't find a song's lyrics then skip and append empty string
    if searched_song is None:
        lyrics_arr.append("")
        continue
        
    # get the lyric
    lyric = searched_song.lyrics
    
    # replace the lyrics newline with ". "
    lyric = lyric.replace("\n", ". ")
    
    # remove initial non-lyrics character:
    # Source: https://thispointer.com/remove-string-before-a-specific-character-in-python/
    # lyric = lyric[lyric.index('.') + 1 :]
    
    # append the processed lyric to the array
    lyrics_arr.append(lyric)

Searching for "Calma - Remix"...
Done.
Searching for "Con Calma"...
Done.
Searching for "Te Vi"...
Done.
Searching for "Otro Trago"...
Done.
Searching for "Callaita"...
Done.
Searching for "Desconocidos"...
Done.
Searching for "HP"...
Done.
Searching for "Secreto"...
Done.
Searching for "LA CANCIÓN"...
Done.
Searching for "Adan y Eva"...
Done.
Searching for "11 PM"...
Done.
Searching for "Soltera (Remix)"...
Done.
Searching for "Con Altura"...
Done.
Searching for "Tal Vez"...
Done.
Searching for "China"...
Done.
Searching for "No Me Conoce - Remix"...
Done.
Searching for "Pa Mí - Remix"...
Done.
Searching for "Ella Quiere Beber - Remix"...
Done.
Searching for "Un Año"...
Done.
Searching for "MIA"...
Done.
Searching for "Dancin (feat. Luvli) - Krono Remix"...
Done.
Searching for "Sunflower - Spider-Man: Into the Spider-Verse"...
Done.
Searching for "7 rings"...
Done.
Searching for "Señorita"...
Done.
Searching for "bad guy"...
Done.
Searching for "Baby Girl (feat. Lalo Ebratt)"...
Done.

In [12]:
# Check length
len(lyrics_arr)

100

In [13]:
# Append lyric columns
lyrics_df['lyrics'] = lyrics_arr

# Replace "" with Nan
lyrics_df = lyrics_df.replace('', np.nan)

# Drop Nan
lyrics_df = lyrics_df.dropna(subset = ["lyrics"])

lyrics_df

,Song,lyrics
0,Calma - Remix,"24 ContributorsCalma (Remix) Lyrics[Letra de ""..."
1,Con Calma,31 ContributorsTranslationsEnglishCon Calma Ly...
2,Te Vi,"30 ContributorsToda (Remix) Lyrics[Letra de ""T..."
3,Otro Trago,"11 ContributorsOtro Trago Lyrics[Letra de ""Otr..."
4,Callaita,44 ContributorsTranslationsEnglishDeutschItali...
...,...,...
95,Sucker,166 ContributorsTranslationsDeutschไทย / Phasa...
96,Cuando Nadie Ve,11 ContributorsCuando Nadie Ve Lyrics[Letra de...
97,Que Le De,77 ContributorsTranslationsDeutschGasolina Lyr...
98,Ponle,"7 ContributorsPonle Lyrics[Letra de ""Ponle""]. ..."


## Bag of words
# Preprocessing Steps

In [14]:
# Create a bag-of-words column
lyrics_df['bag'] = lyrics_df.lyrics.map(lambda t: t.replace('!','').replace('.','').lower().split())
lyrics_df.head()

,Song,lyrics,bag
0,Calma - Remix,"24 ContributorsCalma (Remix) Lyrics[Letra de ""...","[24, contributorscalma, (remix), lyrics[letra,..."
1,Con Calma,31 ContributorsTranslationsEnglishCon Calma Ly...,"[31, contributorstranslationsenglishcon, calma..."
2,Te Vi,"30 ContributorsToda (Remix) Lyrics[Letra de ""T...","[30, contributorstoda, (remix), lyrics[letra, ..."
3,Otro Trago,"11 ContributorsOtro Trago Lyrics[Letra de ""Otr...","[11, contributorsotro, trago, lyrics[letra, de..."
4,Callaita,44 ContributorsTranslationsEnglishDeutschItali...,"[44, contributorstranslationsenglishdeutschita..."


In [15]:
# Create a number of words in each bag of words
lyrics_df['len'] = lyrics_df.bag.map(len)
lyrics_df.head()

,Song,lyrics,bag,len
0,Calma - Remix,"24 ContributorsCalma (Remix) Lyrics[Letra de ""...","[24, contributorscalma, (remix), lyrics[letra,...",478
1,Con Calma,31 ContributorsTranslationsEnglishCon Calma Ly...,"[31, contributorstranslationsenglishcon, calma...",628
2,Te Vi,"30 ContributorsToda (Remix) Lyrics[Letra de ""T...","[30, contributorstoda, (remix), lyrics[letra, ...",945
3,Otro Trago,"11 ContributorsOtro Trago Lyrics[Letra de ""Otr...","[11, contributorsotro, trago, lyrics[letra, de...",649
4,Callaita,44 ContributorsTranslationsEnglishDeutschItali...,"[44, contributorstranslationsenglishdeutschita...",503


In [16]:
# Create a group of bag of words in case we happen to have 2 similar titles
g = lyrics_df.groupby('Song')
g.size()

Song
11 PM                1
7 rings              1
A Través del Vaso    1
Adan y Eva           1
Adictiva             1
                    ..
Verte Ir             1
Ya No Tiene Novio    1
Yo x Ti, Tu x Mi     1
bad guy              1
thank u, next        1
Length: 100, dtype: int64

In [17]:
title_bags = g.bag.aggregate(lambda bag: [w for b in bag.values for w in b])
title_bags

Song
11 PM                [8, contributors11, pm, lyrics[letra, de, "11,...
7 rings              [473, contributorstranslationstürkçeespañolrom...
A Través del Vaso    [6, contributorsa, través, del, vaso, lyrics[l...
Adan y Eva           [18, contributorsadán, y, eva, lyrics[letra, d...
Adictiva             [17, contributorsadictiva, lyrics[letra, de, "...
                                           ...                        
Verte Ir             [17, contributorsverte, ir, lyrics[letra, de, ...
Ya No Tiene Novio    [15, contributorsya, no, tiene, novio, lyrics[...
Yo x Ti, Tu x Mi     [48, contributorstranslationsportuguêsespañoly...
bad guy              [339, contributorstranslationshrvatskitürkçesv...
thank u, next        [398, contributorstranslationstürkçesvenskaesp...
Name: bag, Length: 100, dtype: object

In [18]:
# Compute Term Frequency Matrix
TF = title_bags.apply(lambda bag: pd.Series(bag).value_counts())
TF

,que,te,no,tú,mejor?,ves,por,seas,merece,la,...,malcolm,an,discussions,havin',met,contributorstranslationstürkçesvenskaespañolрусскийportuguêspolski한국어deutschfrançaisnederlands​thank,worried,"so,","now,",yee512embed
Song,,,,,,,,,,,,,,,,,,,,,
11 PM,21.0,16.0,14.0,12.0,12.0,12.0,9.0,9.0,9.0,9.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7 rings,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
A Través del Vaso,5.0,1.0,5.0,3.0,NaN,NaN,3.0,NaN,NaN,7.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Adan y Eva,18.0,9.0,18.0,1.0,NaN,NaN,NaN,NaN,NaN,8.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Adictiva,5.0,8.0,8.0,11.0,NaN,NaN,1.0,NaN,NaN,10.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Verte Ir,34.0,49.0,23.0,33.0,NaN,NaN,6.0,NaN,NaN,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Ya No Tiene Novio,18.0,9.0,30.0,NaN,NaN,NaN,1.0,NaN,NaN,18.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"Yo x Ti, Tu x Mi",16.0,5.0,2.0,44.0,NaN,NaN,79.0,NaN,NaN,7.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
TF_words=TF.T
TF_words

Song,11 PM,7 rings,A Través del Vaso,Adan y Eva,Adictiva,Adiós Amor,Aerolínea Carrillo,Ahora Te Puedes Marchar,Amanece,Amigos Con Derechos,...,Te Vi,Tra Tra Tra Remix - Remix,Tutu,Un Año,Vaina Loca,Verte Ir,Ya No Tiene Novio,"Yo x Ti, Tu x Mi",bad guy,"thank u, next"
que,21.0,NaN,5.0,18.0,5.0,NaN,4.0,15.0,24.0,17.0,...,26.0,29.0,9.0,11.0,36.0,34.0,18.0,16.0,NaN,NaN
te,16.0,NaN,1.0,9.0,8.0,6.0,NaN,17.0,9.0,15.0,...,33.0,3.0,11.0,10.0,2.0,49.0,9.0,5.0,NaN,NaN
no,14.0,2.0,5.0,18.0,8.0,6.0,1.0,9.0,9.0,5.0,...,10.0,5.0,22.0,1.0,20.0,23.0,30.0,2.0,1.0,2.0
tú,12.0,NaN,3.0,1.0,11.0,NaN,NaN,2.0,10.0,1.0,...,18.0,3.0,1.0,3.0,13.0,33.0,NaN,44.0,NaN,NaN
mejor?,12.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
contributorstranslationstürkçesvenskaespañolрусскийportuguêspolski한국어deutschfrançaisnederlands​thank,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
worried,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
"so,",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
"now,",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0


In [20]:
TF_words.index

Index(['que', 'te', 'no', 'tú', 'mejor?', 'ves', 'por', 'seas', 'merece', 'la',
       ...
       'malcolm', 'an', 'discussions', 'havin'', 'met',
       'contributorstranslationstürkçesvenskaespañolрусскийportuguêspolski한국어deutschfrançaisnederlands​thank',
       'worried', 'so,', 'now,', 'yee512embed'],
      dtype='object', length=8996)

In [21]:
#T = lyrics_df.replace('', '?')"
words_count_df=TF.count()
words_count_df




que                                                                                                     74
te                                                                                                      64
no                                                                                                      88
tú                                                                                                      54
mejor?                                                                                                   1
                                                                                                        ..
contributorstranslationstürkçesvenskaespañolрусскийportuguêspolski한국어deutschfrançaisnederlands​thank     1
worried                                                                                                  1
so,                                                                                                      1
now,                                 

In [37]:
# Option 1. Ex. Eliminate unwanted words by code, using the natural language toolkit (import nltk, pip install nltk)

text = "I am very happy to be here today"
tokens = nltk.word_tokenize(text, language="english")
pos_tagged_tokens = nltk.pos_tag(tokens)
pos_tagged_tokens

LookupError: 
**********************************************************************
  Resource [93mpunkt[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt/PY3/english.pickle[0m

  Searched in:
    - '/Users/auroracorzas/nltk_data'
    - '/Users/auroracorzas/anaconda3/nltk_data'
    - '/Users/auroracorzas/anaconda3/share/nltk_data'
    - '/Users/auroracorzas/anaconda3/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
    - ''
**********************************************************************


In [22]:
# Option 2: Manually eliminate the list of unwanted words
words_cleaned_df=TF_words.drop(["que","te","no","un","y", "e", "ni", "o", "u", "pero", "aunque","contributorstranslationstürkçesvenskaespañolрусскийportuguêspolski한국어deutschfrançaisnederlands​thank","so","yee512embed","in","at","on"],axis=0)
words_cleaned_df

Song,11 PM,7 rings,A Través del Vaso,Adan y Eva,Adictiva,Adiós Amor,Aerolínea Carrillo,Ahora Te Puedes Marchar,Amanece,Amigos Con Derechos,...,Te Vi,Tra Tra Tra Remix - Remix,Tutu,Un Año,Vaina Loca,Verte Ir,Ya No Tiene Novio,"Yo x Ti, Tu x Mi",bad guy,"thank u, next"
tú,12.0,NaN,3.0,1.0,11.0,NaN,NaN,2.0,10.0,1.0,...,18.0,3.0,1.0,3.0,13.0,33.0,NaN,44.0,NaN,NaN
mejor?,12.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ves,12.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
por,9.0,NaN,3.0,NaN,1.0,NaN,2.0,4.0,NaN,NaN,...,8.0,1.0,1.0,3.0,9.0,6.0,1.0,79.0,NaN,NaN
seas,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
havin',NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
met,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
worried,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
"so,",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0


In [23]:
TF_words.to_csv("PALCAN.csv")
# Load in csv
words_df = pd.read_csv("PALCAN.csv")
words_df


,Unnamed: 0,11 PM,7 rings,A Través del Vaso,Adan y Eva,Adictiva,Adiós Amor,Aerolínea Carrillo,Ahora Te Puedes Marchar,Amanece,...,Te Vi,Tra Tra Tra Remix - Remix,Tutu,Un Año,Vaina Loca,Verte Ir,Ya No Tiene Novio,"Yo x Ti, Tu x Mi",bad guy,"thank u, next"
0,que,21.0,NaN,5.0,18.0,5.0,NaN,4.0,15.0,24.0,...,26.0,29.0,9.0,11.0,36.0,34.0,18.0,16.0,NaN,NaN
1,te,16.0,NaN,1.0,9.0,8.0,6.0,NaN,17.0,9.0,...,33.0,3.0,11.0,10.0,2.0,49.0,9.0,5.0,NaN,NaN
2,no,14.0,2.0,5.0,18.0,8.0,6.0,1.0,9.0,9.0,...,10.0,5.0,22.0,1.0,20.0,23.0,30.0,2.0,1.0,2.0
3,tú,12.0,NaN,3.0,1.0,11.0,NaN,NaN,2.0,10.0,...,18.0,3.0,1.0,3.0,13.0,33.0,NaN,44.0,NaN,NaN
4,mejor?,12.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8991,contributorstranslationstürkçesvenskaespañolру...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
8992,worried,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
8993,"so,",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
8994,"now,",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0


In [24]:
# Compute IDF vector:  (count non-NaN)
DF = TF.count()
n = len(TF) # Nummber of rows
IDF = np.log(float(n)/DF)
IDF

que                                                                                                     0.301105
te                                                                                                      0.446287
no                                                                                                      0.127833
tú                                                                                                      0.616186
mejor?                                                                                                  4.605170
                                                                                                          ...   
contributorstranslationstürkçesvenskaespañolрусскийportuguêspolski한국어deutschfrançaisnederlands​thank    4.605170
worried                                                                                                 4.605170
so,                                                                                             

In [25]:
# Compute TFIDF matrix:
TFIDF = TF * IDF
TFIDF

,que,te,no,tú,mejor?,ves,por,seas,merece,la,...,malcolm,an,discussions,havin',met,contributorstranslationstürkçesvenskaespañolрусскийportuguêspolski한국어deutschfrançaisnederlands​thank,worried,"so,","now,",yee512embed
Song,,,,,,,,,,,,,,,,,,,,,
11 PM,6.323207,7.140594,1.789667,7.394234,55.262042,31.91112,6.238325,35.208207,41.446532,2.589139,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7 rings,NaN,NaN,0.255667,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
A Través del Vaso,1.505525,0.446287,0.639167,1.848558,NaN,NaN,2.079442,NaN,NaN,2.013775,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Adan y Eva,5.419892,4.016584,2.301001,0.616186,NaN,NaN,NaN,NaN,NaN,2.301457,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Adictiva,1.505525,3.570297,1.022667,6.778048,NaN,NaN,0.693147,NaN,NaN,2.876821,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Verte Ir,10.237573,21.868068,2.940168,20.334143,NaN,NaN,4.158883,NaN,NaN,1.438410,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Ya No Tiene Novio,5.419892,4.016584,3.835001,NaN,NaN,NaN,0.693147,NaN,NaN,5.178277,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"Yo x Ti, Tu x Mi",4.817681,2.231436,0.255667,27.112190,NaN,NaN,54.758627,NaN,NaN,2.013775,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
# Normalize documents to unit length
docmags = np.sqrt((TFIDF*TFIDF).sum(axis=1))
docmags

Song
11 PM                134.772411
7 rings              156.777808
A Través del Vaso     56.702252
Adan y Eva           131.068561
Adictiva              87.156585
                        ...    
Verte Ir             130.814200
Ya No Tiene Novio    179.452426
Yo x Ti, Tu x Mi     163.972209
bad guy              100.446048
thank u, next        217.912499
Length: 100, dtype: float64

In [27]:
normalized_TFIDF = TFIDF.divide(docmags, axis=0)
normalized_TFIDF

,que,te,no,tú,mejor?,ves,por,seas,merece,la,...,malcolm,an,discussions,havin',met,contributorstranslationstürkçesvenskaespañolрусскийportuguêspolski한국어deutschfrançaisnederlands​thank,worried,"so,","now,",yee512embed
Song,,,,,,,,,,,,,,,,,,,,,
11 PM,0.046918,0.052983,0.013279,0.054865,0.41004,0.236778,0.046288,0.261242,0.30753,0.019211,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7 rings,NaN,NaN,0.001631,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
A Través del Vaso,0.026551,0.007871,0.011272,0.032601,NaN,NaN,0.036673,NaN,NaN,0.035515,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Adan y Eva,0.041352,0.030645,0.017556,0.004701,NaN,NaN,NaN,NaN,NaN,0.017559,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Adictiva,0.017274,0.040964,0.011734,0.077769,NaN,NaN,0.007953,NaN,NaN,0.033007,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Verte Ir,0.078260,0.167169,0.022476,0.155443,NaN,NaN,0.031792,NaN,NaN,0.010996,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Ya No Tiene Novio,0.030202,0.022382,0.021371,NaN,NaN,NaN,0.003863,NaN,NaN,0.028856,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"Yo x Ti, Tu x Mi",0.029381,0.013609,0.001559,0.165346,NaN,NaN,0.333951,NaN,NaN,0.012281,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
# Check normalization
(normalized_TFIDF*normalized_TFIDF).sum(axis=1)

Song
11 PM                1.0
7 rings              1.0
A Través del Vaso    1.0
Adan y Eva           1.0
Adictiva             1.0
                    ... 
Verte Ir             1.0
Ya No Tiene Novio    1.0
Yo x Ti, Tu x Mi     1.0
bad guy              1.0
thank u, next        1.0
Length: 100, dtype: float64

In [29]:
# Matrix multiply
new_n_TFIDF = normalized_TFIDF.copy()
new_n_TFIDF = new_n_TFIDF.fillna(0.0)

In [30]:
# Calculate Cosine Similarity matrix
cos_sim = new_n_TFIDF.dot(new_n_TFIDF.transpose())
cos_sim

Song,11 PM,7 rings,A Través del Vaso,Adan y Eva,Adictiva,Adiós Amor,Aerolínea Carrillo,Ahora Te Puedes Marchar,Amanece,Amigos Con Derechos,...,Te Vi,Tra Tra Tra Remix - Remix,Tutu,Un Año,Vaina Loca,Verte Ir,Ya No Tiene Novio,"Yo x Ti, Tu x Mi",bad guy,"thank u, next"
Song,,,,,,,,,,,,,,,,,,,,,
11 PM,1.000000,0.001181,0.011987,0.014538,0.042872,0.038232,0.015406,0.021469,0.024593,0.025738,...,0.056000,0.007477,0.019383,0.013719,0.028001,0.063158,0.010539,0.034110,0.000388,0.000041
7 rings,0.001181,1.000000,0.000099,0.002882,0.002923,0.000102,0.000042,0.000045,0.000336,0.000043,...,0.002263,0.000093,0.005336,0.000067,0.001097,0.000492,0.000957,0.001034,0.112625,0.083916
A Través del Vaso,0.011987,0.000099,1.000000,0.014527,0.029690,0.038360,0.028206,0.017942,0.030301,0.022728,...,0.043237,0.010120,0.015314,0.023118,0.021708,0.040684,0.025844,0.031714,0.000141,0.000914
Adan y Eva,0.014538,0.002882,0.014527,1.000000,0.025998,0.031794,0.022423,0.021101,0.040871,0.066837,...,0.053761,0.009248,0.031587,0.018002,0.020482,0.046120,0.020479,0.033845,0.003804,0.002350
Adictiva,0.042872,0.002923,0.029690,0.025998,1.000000,0.056269,0.025365,0.034370,0.070931,0.038564,...,0.055523,0.013546,0.029281,0.050416,0.062849,0.123241,0.022038,0.044844,0.001008,0.002554
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Verte Ir,0.063158,0.000492,0.040684,0.046120,0.123241,0.052187,0.025869,0.050941,0.166430,0.046198,...,0.126129,0.028590,0.036659,0.061379,0.054213,1.000000,0.041551,0.078440,0.000708,0.000738
Ya No Tiene Novio,0.010539,0.000957,0.025844,0.020479,0.022038,0.025603,0.014450,0.037942,0.034716,0.022848,...,0.046576,0.026370,0.049538,0.126688,0.025842,0.041551,1.000000,0.023739,0.000302,0.000657
"Yo x Ti, Tu x Mi",0.034110,0.001034,0.031714,0.033845,0.044844,0.021569,0.028692,0.037604,0.039294,0.025052,...,0.065095,0.010056,0.039867,0.034010,0.092104,0.078440,0.023739,1.000000,0.000064,0.000038


In [31]:
# Get rid of the diagonal line of matrix since we want to see which songs are most similarity with others
new_cos_sim = cos_sim - np.identity(len(new_n_TFIDF))
new_cos_sim

Song,11 PM,7 rings,A Través del Vaso,Adan y Eva,Adictiva,Adiós Amor,Aerolínea Carrillo,Ahora Te Puedes Marchar,Amanece,Amigos Con Derechos,...,Te Vi,Tra Tra Tra Remix - Remix,Tutu,Un Año,Vaina Loca,Verte Ir,Ya No Tiene Novio,"Yo x Ti, Tu x Mi",bad guy,"thank u, next"
Song,,,,,,,,,,,,,,,,,,,,,
11 PM,0.000000,1.180990e-03,1.198654e-02,1.453814e-02,4.287164e-02,0.038232,0.015406,0.021469,0.024593,0.025738,...,0.056000,0.007477,0.019383,0.013719,0.028001,6.315754e-02,1.053897e-02,3.410989e-02,3.883020e-04,4.111946e-05
7 rings,0.001181,2.220446e-16,9.892162e-05,2.882150e-03,2.922982e-03,0.000102,0.000042,0.000045,0.000336,0.000043,...,0.002263,0.000093,0.005336,0.000067,0.001097,4.922945e-04,9.574324e-04,1.034275e-03,1.126251e-01,8.391605e-02
A Través del Vaso,0.011987,9.892162e-05,4.440892e-16,1.452712e-02,2.969036e-02,0.038360,0.028206,0.017942,0.030301,0.022728,...,0.043237,0.010120,0.015314,0.023118,0.021708,4.068443e-02,2.584425e-02,3.171398e-02,1.412671e-04,9.142180e-04
Adan y Eva,0.014538,2.882150e-03,1.452712e-02,-2.220446e-16,2.599808e-02,0.031794,0.022423,0.021101,0.040871,0.066837,...,0.053761,0.009248,0.031587,0.018002,0.020482,4.611971e-02,2.047859e-02,3.384453e-02,3.803758e-03,2.349697e-03
Adictiva,0.042872,2.922982e-03,2.969036e-02,2.599808e-02,-4.440892e-16,0.056269,0.025365,0.034370,0.070931,0.038564,...,0.055523,0.013546,0.029281,0.050416,0.062849,1.232408e-01,2.203843e-02,4.484359e-02,1.008294e-03,2.553928e-03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Verte Ir,0.063158,4.922945e-04,4.068443e-02,4.611971e-02,1.232408e-01,0.052187,0.025869,0.050941,0.166430,0.046198,...,0.126129,0.028590,0.036659,0.061379,0.054213,-8.881784e-16,4.155077e-02,7.843954e-02,7.078239e-04,7.377755e-04
Ya No Tiene Novio,0.010539,9.574324e-04,2.584425e-02,2.047859e-02,2.203843e-02,0.025603,0.014450,0.037942,0.034716,0.022848,...,0.046576,0.026370,0.049538,0.126688,0.025842,4.155077e-02,-2.220446e-16,2.373878e-02,3.015014e-04,6.569288e-04
"Yo x Ti, Tu x Mi",0.034110,1.034275e-03,3.171398e-02,3.384453e-02,4.484359e-02,0.021569,0.028692,0.037604,0.039294,0.025052,...,0.065095,0.010056,0.039867,0.034010,0.092104,7.843954e-02,2.373878e-02,2.220446e-16,6.368738e-05,3.823248e-05


In [32]:
# Sort from the most similar to less similar songs by lyrics
new_cos_sim.max().sort_values(ascending=False)

Song
Promises (with Sam Smith)                          0.897210
Taki Taki (feat. Selena Gomez, Ozuna & Cardi B)    0.897210
Dancing With A Stranger (with Normani)             0.891758
I Don't Care (with Justin Bieber)                  0.888041
Bohemian Rhapsody - Remastered 2011                0.835271
                                                     ...   
Brillas                                            0.057987
Con Altura                                         0.055103
Tra Tra Tra Remix - Remix                          0.055049
Aerolínea Carrillo                                 0.052981
Prometiste                                         0.038629
Length: 100, dtype: float64

In [33]:
# Song similarity pairs
new_cos_sim.idxmax()

Song
11 PM                 Nunca Es Suficiente
7 rings                              BEBE
A Través del Vaso    Que Mas Pues - Remix
Adan y Eva                        Tal Vez
Adictiva                        Con Calma
                             ...         
Verte Ir                          Secreto
Ya No Tiene Novio            Desconocidos
Yo x Ti, Tu x Mi            Pa Mí - Remix
bad guy                              BEBE
thank u, next                        Sola
Length: 100, dtype: object

In [34]:
# Output song-pairs that are similarity with each other 
output = pd.DataFrame(new_cos_sim.idxmax())
output.to_csv('song_similarity.csv', index=False)